In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, f_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [2]:
df_test = pd.read_csv('dftest_cleaned.csv')
df_test.drop('Unnamed: 0', axis=1)
features = df_test[['hourly_basis', 'Age', 'category', 'amount_qt', 'merchant', 'job','transaction_id']]
target = df_test.is_fraud

In [3]:
# NOTE: Make sure that the outcome column is labeled 'target' in the data file
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, target, random_state=1)

In [8]:
# Average CV score on the training set was: 0.9982671062316886
exported_pipeline = make_pipeline(
    SelectPercentile(score_func=f_classif, percentile=44),
    VarianceThreshold(threshold=0.0001),
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="entropy", max_depth=9, min_samples_leaf=1, min_samples_split=14)),
    BernoulliNB(alpha=0.001, fit_prior=True)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)